In [1]:
## this file considers feature selection technique as label

import pandas as pd
import seaborn as sb
import math
import numpy as np
import os
import re
import jenkspy
from ast import literal_eval

import matplotlib.pyplot as plt

In [2]:
import pandas as pd 
import numpy as np 
from sklearn.preprocessing import LabelEncoder

from sklearn import preprocessing
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import cross_val_score, ShuffleSplit
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, train_test_split

from sklearn import preprocessing

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)


In [3]:
df1 = pd.read_csv("./Datafiles/df_mfeatures.csv")
df2 = pd.read_csv("./Datafiles/df_ensemble_allfeatures.csv")

binDF = pd.read_csv("./Datafiles/BinnedMetaFeatures.csv")

In [4]:
df_new = df1[['File', 'Completeness', 'Conciseness', 'cor.mean', 'cov.mean',
       'eigenvalues.mean', 'g_mean.mean', 'h_mean.mean', 'iq_range.mean', 'kurtosis.mean', 
       'mad.mean', 'max.mean', 'mean.mean', 'median.mean', 'min.mean', 'nr_cor_attr',
       'nr_norm', 'nr_outliers', 'range.mean', 'sd.mean', 'skewness.mean',
       'sparsity.mean', 't_mean.mean',   'var.mean', 'ClassImbRatio',
       'ClassOverlapPerc', 'OutlierPerc', 'attr_to_inst', 'inst_to_attr', 'nr_attr',
        'nr_bin',  'nr_inst', 'nr_num', 'attr_conc.mean', 'attr_ent.mean', 'ena', 'nUnique' ,'snr.mean', 'cEntropy','LabelIssuesPerc']] #SyntaxAccuracy removed

df_imp_subset = df1[['File', 'cov.mean',
       'eigenvalues.mean', 'iq_range.mean',
        'kurtosis.mean', 'inst_to_attr',
        'skewness.mean', 'mean.mean',
       'sparsity.mean','nr_inst', 'nr_num',
        'attr_conc.mean', 'attr_ent.mean', 
        'ena', 'snr.mean', 'cEntropy','LabelIssuesPerc']] #SyntaxAccuracy removed

df_bin_new = binDF[['Completeness_bins', 'Conciseness_bins',  'cor.mean_bins', 'cov.mean_bins', 
              'eigenvalues.mean_bins', 'g_mean.mean_bins', 'h_mean.mean_bins', 'iq_range.mean_bins', 'kurtosis.mean_bins',
              'mad.mean_bins', 'max.mean_bins',  'mean.mean_bins', 'median.mean_bins',  'min.mean_bins', 'nr_cor_attr_bins', 
              'nr_norm_bins', 'nr_outliers_bins', 'range.mean_bins', 'sd.mean_bins', 'skewness.mean_bins',
              'sparsity.mean_bins', 't_mean.mean_bins', 'var.mean_bins', 'LabelIssues_bins', 'ClassImbRatio_bins', 
              'OutlierPerc_bins', 'ClassOverlapPerc_bins', 'attr_to_inst_bins', 'inst_to_attr_bins', 'nr_attr_bins',
              'nr_inst_bins', 'nr_num_bins', 'nr_bin_bins', 'attr_conc.mean_bins', 'cEntropy_bins', 'ena_bins', 'snr.mean_bins', 'nUnique_bins','attr_ent.mean_bins', 'File']] #'SyntaxAccuracy_bins' removed

In [5]:
(df_imp_subset).shape

(173, 17)

Majority voting calculation

In [6]:
import pandas as pd
import os
from collections import Counter
from glob import glob

path = os.getcwd()+'/optimalalphabeta/dearr_cont/'  
csv_files = glob(os.path.join(path, 'dearr*.csv'))

labels_dict = {}

for file in csv_files:
    df = pd.read_csv(file)
    
    for idx, row in df.iterrows():
        dataset_id = row['File']  
        best_fs = row['FeatureAlgo']     
        
        if dataset_id not in labels_dict:
            labels_dict[dataset_id] = []
        labels_dict[dataset_id].append(best_fs)

majority_labels = {
    dataset: Counter(fs_list).most_common(1)[0][0]
    for dataset, fs_list in labels_dict.items()
}

majority_df = pd.DataFrame(list(majority_labels.items()), columns=['File', 'FeatureAlgo'])
majority_df.to_csv('majority_voted_labels.csv', index=False)

print(" Majority vote labels saved to 'majority_voted_labels.csv'")

 Majority vote labels saved to 'majority_voted_labels.csv'


Majority voting based on cluster

In [7]:
import pandas as pd
import os
from collections import Counter
from glob import glob

path = os.getcwd()+'/optimalalphabeta/clusteredearr/'  # replace with the path to your 36 CSVs
csv_files = glob(os.path.join(path, 'dearr*.csv'))

labels_dict = {}

for file in csv_files:
    df = pd.read_csv(file)
    
    for idx, row in df.iterrows():
        dataset_id = row['File']  # replace 'Dataset' with your actual column name
        best_fs = row['FeatureAlgo']     # replace 'Best_FS' with your actual column name
        
        if dataset_id not in labels_dict:
            labels_dict[dataset_id] = []
        labels_dict[dataset_id].append(best_fs)

majority_labels = {
    dataset: Counter(fs_list).most_common(1)[0][0]
    for dataset, fs_list in labels_dict.items()
}

majority_df = pd.DataFrame(list(majority_labels.items()), columns=['File', 'FeatureAlgo'])
majority_df.to_csv('majority_voted_labels_clusteredearr.csv', index=False)

In [8]:
#Merging two dataframes to add target feature
df1 = pd.read_csv('majority_voted_labels_clusteredearr.csv')

df2 = pd.read_csv('./Datafiles/meta-features-cont.csv')
merged_df = pd.merge(df1, df2, on='File', how='inner')  # use 'left', 'right', or 'outer' as needed

merged_df.to_csv("majorityvoteclusteredearrdataset.csv",index=False)

In [9]:
fs_mapping = {
    'MI': ('Uni', 'Information'),
    'GR': ('Uni', 'Information'),
    'fcbf': ('Multi', 'Information'),
    'chisquare': ('Uni', 'Dependency'),
    'mRMR': ('Multi', 'Dependency'),
    'cfs': ('Multi', 'Dependency'),
    'relief': ('Uni', 'Distance'),
    'relieff': ('Uni', 'Distance'),
    'multisurf': ('Multi', 'Distance')
}

def get_setting(algo):
    return fs_mapping.get(algo, ('Unknown', 'Unknown'))  # default if not found


merged_df[['FS_Setting', 'FS_Type']] = merged_df['FeatureAlgo'].apply(lambda x: pd.Series(get_setting(x)))

Applying feature selection 

In [28]:
import pandas as pd
import numpy as np

def correlation_filter(df, threshold=0.9):
    corr_matrix = df.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    return df.drop(columns=to_drop)
df1 = pd.read_csv('majority_voted_labels_clusteredearr.csv')

df2 = pd.read_csv('./Datafiles/meta-features-cont.csv')
# Separate the 'File' column
file_col = df2['File']

# Apply correlation filter to the rest of the columns
X_filtered = correlation_filter(df2.drop(columns=['File']))

X_filtered['File'] = file_col

X_filtered = pd.merge(X_filtered, df1 , on='File', how='inner') 

In [29]:
X_filtered.columns

Index(['Completeness', 'Conciseness', 'cor.mean', 'cov.mean',
       'eigenvalues.mean', 'kurtosis.mean', 'max.mean', 'nr_norm',
       'nr_outliers', 'sparsity.mean', 'ClassImbRatio', 'ClassOverlapPerc',
       'OutlierPerc', 'attr_to_inst', 'inst_to_attr', 'nr_bin', 'nr_inst',
       'attr_conc.mean', 'attr_ent.mean', 'ena', 'nUnique', 'snr.mean',
       'cEntropy', 'LabelIssuesPerc', 'File', 'FeatureAlgo'],
      dtype='object')

Feature selection method 2

In [11]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

X = merged_df.drop(['File', 'FeatureAlgo','FS_Setting','FS_Type'], axis=1)
X = X.apply(le.fit_transform)
y=merged_df['FeatureAlgo']
clf = RandomForestClassifier(random_state=42)
clf.fit(X, y)

# Select features with importance greater than the mean importance
selector = SelectFromModel(clf, threshold='mean', prefit=True)
X_selected = selector.transform(X)

# Get feature names
selected_features = X.columns[selector.get_support()]
print("Selected Features based on model importance:")
print(selected_features.tolist())


Selected Features based on model importance:
['kurtosis.mean', 'sparsity.mean', 'OutlierPerc', 'attr_to_inst', 'inst_to_attr', 'nr_attr', 'nr_inst', 'nr_num', 'attr_ent.mean', 'ena', 'snr.mean', 'cEntropy', 'LabelIssuesPerc']


In [12]:
all_features = list(set(selected_features.tolist()).union(set(X_filtered.columns)))
common_features = list(set(selected_features.tolist()).intersection(set(X_filtered.columns)))
len(common_features)

11

In [13]:
imp_features = set(df_imp_subset.columns) - set(['File'])
common_features = list(set(selected_features.tolist()).intersection(set(X_filtered.columns)).intersection(set(df_imp_subset.columns)))
all_features = list(set(selected_features.tolist()).union(set(X_filtered.columns)).union(set(imp_features)))
all_features = list(set(all_features)-set(['FeatureAlgo','File']))

In [14]:
len(common_features)

9

In [15]:
from sklearn.tree import DecisionTreeClassifier, _tree
import numpy as np
import pandas as pd

def get_rules_with_support_confidence(tree_model, feature_names, X, y):
    tree_ = tree_model.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    
    paths = []
    rules = []

    def recurse(node, path, conds):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            left_path = path.copy()
            left_path.append(f"({name} <= {threshold:.3f})")
            recurse(tree_.children_left[node], left_path, conds)

            right_path = path.copy()
            right_path.append(f"({name} > {threshold:.3f})")
            recurse(tree_.children_right[node], right_path, conds)
        else:
            path_str = " and ".join(path)
            support = tree_.n_node_samples[node] / tree_.n_node_samples[0]
            value = tree_.value[node][0]
            predicted_class = np.argmax(value)
            confidence = value[predicted_class] / np.sum(value)
            rules.append((path_str, predicted_class, support, confidence))

    recurse(0, [], rules)

    return pd.DataFrame(rules, columns=["Rule", "Predicted_Class", "Support", "Confidence"])

In [30]:
#Without clustering - all-features

df1 = pd.read_csv('majority_voted_labels_clusteredearr.csv')

df2 = pd.read_csv('./Datafiles/meta-features-cont.csv')

#merged_df = pd.merge(df1, df2, on='File', how='inner')  # use 'left', 'right', or 'outer' as needed

le = LabelEncoder()
#X = merged_df.drop(['File','FS_Setting','FS_Type','FeatureAlgo'], axis=1)
X = merged_df[common_features]
df_new
X = X.apply(le.fit_transform)

y = merged_df['FeatureAlgo'] # Encode both
label_encoder = LabelEncoder().fit(y)
y = label_encoder.transform(y)
list(label_encoder.classes_)


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.8)

# Train a simple tree (can be one of your hierarchical models)
clf = DecisionTreeClassifier(max_depth=4, random_state=0)
clf.fit(X_train, y_train)  # y_train can be FS_Type or FS_Setting

# Get readable rules with metrics
rule_df = get_rules_with_support_confidence(clf, feature_names=X_train.columns.tolist(), X=X_train, y=y_train)

# Display top rules
print(rule_df.sort_values(by='Confidence', ascending=False).head(10))

# Evaluate
from sklearn.metrics import classification_report

y_type_pred = clf.predict(X_test)

print("Feature_Algo Report (all features):")
print(classification_report(y_test, y_type_pred))


                                                 Rule  Predicted_Class  \
0        (nr_inst <= 51.500) and (cEntropy <= 27.000)                2   
6   (nr_inst > 51.500) and (snr.mean <= 121.500) a...                1   
8   (nr_inst > 51.500) and (snr.mean <= 121.500) a...                2   
9   (nr_inst > 51.500) and (snr.mean > 121.500) an...                3   
11  (nr_inst > 51.500) and (snr.mean > 121.500) an...                2   
5   (nr_inst > 51.500) and (snr.mean <= 121.500) a...                2   
2   (nr_inst <= 51.500) and (cEntropy > 27.000) an...                0   
3   (nr_inst <= 51.500) and (cEntropy > 27.000) an...                2   
10  (nr_inst > 51.500) and (snr.mean > 121.500) an...                0   
1   (nr_inst <= 51.500) and (cEntropy > 27.000) an...                0   

     Support  Confidence  
0   0.036232    1.000000  
6   0.007246    1.000000  
8   0.021739    1.000000  
9   0.036232    1.000000  
11  0.014493    1.000000  
5   0.391304    0.88888

/home/d19125691/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/d19125691/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/d19125691/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
int_to_label = {i: label for i, label in enumerate(label_encoder.classes_)}
print(int_to_label)

{0: 'GR', 1: 'MI', 2: 'chisquare', 3: 'fcbf', 4: 'relief'}


In [25]:
#With clustering
le = LabelEncoder()
#X = merged_df.drop(['File','FS_Setting','FS_Type','FeatureAlgo'], axis=1)
X = merged_df[common_features]
X = X.apply(le.fit_transform)

y = merged_df['FeatureAlgo'] # Encode both

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.8)

# Train a simple tree (can be one of your hierarchical models)
clf = DecisionTreeClassifier(max_depth=4, random_state=0)
clf.fit(X_train, y_train)  # y_train can be FS_Type or FS_Setting

# Get readable rules with metrics
rule_df = get_rules_with_support_confidence(clf, feature_names=X_train.columns.tolist(), X=X_train, y=y_train)

# Display top rules
print(rule_df.sort_values(by='Confidence', ascending=False).head(10))


                                                 Rule  Predicted_Class  \
0        (nr_inst <= 51.500) and (cEntropy <= 27.000)                2   
6   (nr_inst > 51.500) and (snr.mean <= 121.500) a...                1   
8   (nr_inst > 51.500) and (snr.mean <= 121.500) a...                2   
9   (nr_inst > 51.500) and (snr.mean > 121.500) an...                3   
11  (nr_inst > 51.500) and (snr.mean > 121.500) an...                2   
5   (nr_inst > 51.500) and (snr.mean <= 121.500) a...                2   
2   (nr_inst <= 51.500) and (cEntropy > 27.000) an...                0   
3   (nr_inst <= 51.500) and (cEntropy > 27.000) an...                2   
10  (nr_inst > 51.500) and (snr.mean > 121.500) an...                0   
1   (nr_inst <= 51.500) and (cEntropy > 27.000) an...                0   

     Support  Confidence  
0   0.036232    1.000000  
6   0.007246    1.000000  
8   0.021739    1.000000  
9   0.036232    1.000000  
11  0.014493    1.000000  
5   0.391304    0.88888

In [19]:
rule_df.shape

(12, 4)

In [27]:
#Without clustering
le = LabelEncoder()
#X = merged_df.drop(['File','FS_Setting','FS_Type','FeatureAlgo'], axis=1)
X = merged_df[common_features]
X = X.apply(le.fit_transform)

y = merged_df['FeatureAlgo']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.8)

# Train a simple tree (can be one of your hierarchical models)
clf = DecisionTreeClassifier(max_depth=4, random_state=0)
clf.fit(X_train, y_train)  # y_train can be FS_Type or FS_Setting

# Get readable rules with metrics
rule_df = get_rules_with_support_confidence(clf, feature_names=X_train.columns.tolist(), X=X_train, y=y_train)

# Display top rules
print(rule_df.sort_values(by='Confidence', ascending=False).head(10))


                                                 Rule  Predicted_Class  \
0        (nr_inst <= 51.500) and (cEntropy <= 27.000)                2   
6   (nr_inst > 51.500) and (snr.mean <= 121.500) a...                1   
8   (nr_inst > 51.500) and (snr.mean <= 121.500) a...                2   
9   (nr_inst > 51.500) and (snr.mean > 121.500) an...                3   
11  (nr_inst > 51.500) and (snr.mean > 121.500) an...                2   
5   (nr_inst > 51.500) and (snr.mean <= 121.500) a...                2   
2   (nr_inst <= 51.500) and (cEntropy > 27.000) an...                0   
3   (nr_inst <= 51.500) and (cEntropy > 27.000) an...                2   
10  (nr_inst > 51.500) and (snr.mean > 121.500) an...                0   
1   (nr_inst <= 51.500) and (cEntropy > 27.000) an...                0   

     Support  Confidence  
0   0.036232    1.000000  
6   0.007246    1.000000  
8   0.021739    1.000000  
9   0.036232    1.000000  
11  0.014493    1.000000  
5   0.391304    0.88888

In [21]:

# You can chain oversamplers in a pipeline
over_sampler1 = RandomOverSampler(random_state=42)
smote1 = SMOTE(sampling_strategy='not majority', random_state=42)

# Step 1: Random oversample small classes
X_ros, y_ros = over_sampler1.fit_resample(X_train, y_train)

# Step 2: Apply SMOTE to remaining minority classes
X_balanced, y_balanced = smote1.fit_resample(X_ros, y_ros)

clf5 = clf = DecisionTreeClassifier(max_depth=4, random_state=0)
#(class_weight='balanced', random_state=42)
clf5.fit(X_balanced, y_balanced)

y_type_pred = clf5.predict(X_test)

print("Feature_Algo Report (common features):")
print(classification_report(y_test, y_type_pred))

NameError: name 'RandomOverSampler' is not defined

In [ ]:
#Without clustering
clf.fit(X_train, y_train)  # y_train can be FS_Type or FS_Setting

# Evaluate
from sklearn.metrics import classification_report

y_type_pred = clf.predict(X_test)

print("Feature_Algo Report (all features):")
print(classification_report(y_test, y_type_pred))

In [ ]:
#Without clustering
clf.fit(X_train, y_train)  # y_train can be FS_Type or FS_Setting

# Evaluate
from sklearn.metrics import classification_report

y_type_pred = clf.predict(X_test)

print("Feature_Algo Report (common features):")
print(classification_report(y_test, y_type_pred))

In [ ]:
rule_df.to_csv("rules.csv",index=False)

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

X = merged_df.drop(['File', 'FeatureAlgo'], axis=1)
X = X.apply(le.fit_transform)
y=merged_df['FeatureAlgo']
clf = RandomForestClassifier(random_state=42)
clf.fit(X, y)

# Select features with importance greater than the mean importance
selector = SelectFromModel(clf, threshold='mean', prefit=True)
X_selected = selector.transform(X)

# Get feature names
selected_features = X.columns[selector.get_support()]
print("Selected Features based on model importance:")
print(selected_features.tolist())


In [ ]:
all_features = list(set(selected_features.tolist()).union(set(X_filtered.columns)))
common_features = list(set(selected_features.tolist()).intersection(set(X_filtered.columns)))

In [ ]:
def calcu_scores(df):
    scores = []
    #labels = df['FS_Setting_encoded']
    labels = df['FS_Type_encoded']
    #labels = df['FeatureAlgo']
    features = df.drop(['FeatureAlgo', 'File'], axis=1)

    le = LabelEncoder()
    features = features.apply(le.fit_transform)
    X_train, X_test, y_train, y_test = train_test_split(features, labels, random_state=42, train_size=0.8)

    param_grid_mlp = {
        'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
        'solver': ['sgd', 'adam'],
        'alpha': [0.0001, 0.05],
        'max_iter': [100, 200, 500]
    }

    param_grid_lr = {
        'penalty': ['l1', 'l2'],
        'C': [0.01, 0.1, 1, 10, 100],
        'solver': ['saga']
    }

    param_grid_svc = {
        'C': [0.01, 0.1, 1, 10, 100],
        'max_iter': [1000, 2000, 3000]
    }

    param_grid_dt = {'max_features': ['auto', 'sqrt', 'log2'],
              'ccp_alpha': [0.1, .01, .001],
              'max_depth' : [5, 7, 9],
              'criterion' :['gini', 'entropy']
             }

    mlp_clf = GridSearchCV(MLPClassifier(random_state=1), param_grid_mlp, cv=5)
    lr_clf = GridSearchCV(LogisticRegression(random_state=0, max_iter=1000), param_grid_lr, cv=5)
    dt_clf = GridSearchCV(DecisionTreeClassifier(), param_grid_dt, cv=5)
    svc_clf = GridSearchCV(LinearSVC(random_state=0), param_grid_svc, cv=5)

    mlp_clf.fit(X_train, y_train)
    lr_clf.fit(X_train, y_train)
    dt_clf.fit(X_train, y_train)
    svc_clf.fit(X_train, y_train)

    # Predictions
    mlp_predicted = mlp_clf.predict(X_test)
    lr_predicted = lr_clf.predict(X_test)
    dt_predicted = dt_clf.predict(X_test)
    svc_predicted = svc_clf.predict(X_test)

    # Append F1 scores
    for clf_name, predicted in zip(
        ['MLP', 'LR', 'RF', 'SVC'],
        [mlp_predicted, lr_predicted, dt_predicted, svc_predicted]
    ):
        scores.append(f1_score(predicted, y_test, average='macro'))
        scores.append(f1_score(predicted, y_test, average='micro'))
        scores.append(f1_score(predicted, y_test, average='weighted'))

    return scores

In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Prepare X and multiple y targets
X = merged_df.drop(['File','FeatureAlgo'], axis=1)
X = X.apply(le.fit_transform)

y = merged_df['FeatureAlgo'] # Encode both

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.8)

# Base model
base_clf3 = RandomForestClassifier()

# Train a simple tree (can be one of your hierarchical models)
clf = DecisionTreeClassifier(max_depth=4, random_state=0)
base_clf3.fit(X_train, y_train)  # y_train can be FS_Type or FS_Setting

# Evaluate
from sklearn.metrics import classification_report

y_type_pred = base_clf3.predict(X_test)

print("Feature_Algo Report (common features):")
print(classification_report(y_test, y_type_pred))


In [ ]:
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.pipeline import Pipeline

# You can chain oversamplers in a pipeline
over_sampler = RandomOverSampler(random_state=42)
smote = SMOTE(sampling_strategy='not majority', random_state=42)

# Step 1: Random oversample small classes
X_ros, y_ros = over_sampler.fit_resample(X_train, y_train)

# Step 2: Apply SMOTE to remaining minority classes
X_balanced, y_balanced = smote.fit_resample(X_ros, y_ros)

from sklearn.ensemble import RandomForestClassifier
clf4 = RandomForestClassifier(class_weight='balanced', random_state=42)
clf4.fit(X_balanced, y_balanced)

y_type_pred = clf4.predict(X_test)

print("Feature_Algo Report (common features):")
print(classification_report(y_test, y_type_pred))


In [ ]:
X_balanced.shape

In [ ]:
import pandas as pd

fs_mapping = {
    'MI': ('Uni', 'Information'),
    'GR': ('Uni', 'Information'),
    'fcbf': ('Multi', 'Information'),
    'chisquare': ('Uni', 'Dependency'),
    'mRMR': ('Multi', 'Dependency'),
    'cfs': ('Multi', 'Dependency'),
    'relief': ('Uni', 'Distance'),
    'relieff': ('Uni', 'Distance'),
    'multisurf': ('Multi', 'Distance')
}

def get_setting(algo):
    return fs_mapping.get(algo, ('Unknown', 'Unknown'))  # default if not found


merged_df[['FS_Setting', 'FS_Type']] = merged_df['FeatureAlgo'].apply(lambda x: pd.Series(get_setting(x)))

In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Prepare X and multiple y targets
X = merged_df.drop(['File','FS_Setting','FS_Type','FeatureAlgo'], axis=1)
X = X.apply(le.fit_transform)

y = merged_df['FS_Type'] # Encode both

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.8)

# Base model
base_clf4 = RandomForestClassifier()

# Train a simple tree (can be one of your hierarchical models)
clf = DecisionTreeClassifier(max_depth=4, random_state=0)
base_clf4.fit(X_train, y_train)  # y_train can be FS_Type or FS_Setting

# Evaluate
from sklearn.metrics import classification_report

y_type_pred = base_clf4.predict(X_test)

print("Feature_Algo Report (common features):")
print(classification_report(y_test, y_type_pred))

In [ ]:
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.pipeline import Pipeline

# Prepare X and multiple y targets
X = merged_df.drop(['File','FS_Setting','FS_Type','FeatureAlgo'], axis=1)
X = X.apply(le.fit_transform)

y = merged_df['FeatureAlgo'] # Encode both

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.8)

# You can chain oversamplers in a pipeline
over_sampler1 = RandomOverSampler(random_state=42)
smote1 = SMOTE(sampling_strategy='not majority', random_state=42)

# Step 1: Random oversample small classes
X_ros, y_ros = over_sampler1.fit_resample(X_train, y_train)

# Step 2: Apply SMOTE to remaining minority classes
X_balanced, y_balanced = smote1.fit_resample(X_ros, y_ros)

from sklearn.ensemble import RandomForestClassifier
clf5 = clf = DecisionTreeClassifier(max_depth=4, random_state=0)
#(class_weight='balanced', random_state=42)
clf5.fit(X_balanced, y_balanced)

y_type_pred = clf5.predict(X_test)

print("Feature_Algo Report (common features):")
print(classification_report(y_test, y_type_pred))


In [ ]:
from sklearn.tree import DecisionTreeClassifier, _tree
import numpy as np
import pandas as pd

def get_rules_with_support_confidence(tree_model, feature_names, X, y):
    tree_ = tree_model.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    
    paths = []
    rules = []

    def recurse(node, path, conds):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            left_path = path.copy()
            left_path.append(f"({name} <= {threshold:.3f})")
            recurse(tree_.children_left[node], left_path, conds)

            right_path = path.copy()
            right_path.append(f"({name} > {threshold:.3f})")
            recurse(tree_.children_right[node], right_path, conds)
        else:
            path_str = " and ".join(path)
            support = tree_.n_node_samples[node] / tree_.n_node_samples[0]
            value = tree_.value[node][0]
            predicted_class = np.argmax(value)
            confidence = value[predicted_class] / np.sum(value)
            rules.append((path_str, predicted_class, support, confidence))

    recurse(0, [], rules)

    return pd.DataFrame(rules, columns=["Rule", "Predicted_Class", "Support", "Confidence"])

In [ ]:
# Prepare X and multiple y targets
X = merged_df.drop(['File','FS_Setting','FS_Type','FeatureAlgo'], axis=1)
X = X.apply(le.fit_transform)

y = merged_df['FS_Type'] # Encode both

# Train a simple tree (can be one of your hierarchical models)
clf = DecisionTreeClassifier(max_depth=4, random_state=0)
clf.fit(X_train, y_train)  # y_train can be FS_Type or FS_Setting

# Get readable rules with metrics
rule_df = get_rules_with_support_confidence(clf, feature_names=X_train.columns.tolist(), X=X_train, y=y_train)

# Display top rules
print(rule_df.sort_values(by='Confidence', ascending=False).head(10))


In [ ]:
merged_df['FS_Type']